### json 불러와서 저장하기

In [11]:
import numpy as np
import pandas as pd
import json
import jpype
import glob
from random import shuffle
from bs4 import BeautifulSoup as bs

data_path = "data"
#data_path = "data(test)"

file_list = glob.glob("%s/*.json" % data_path)
shuffle(file_list)

json_train=[]
for json_file_name in file_list:
    print json_file_name
    json_file = json.loads(open(json_file_name).read())
    json_train += json_file["articles"]

data/7.json
data/3.json
data/15.json
data/17.json
data/11.json
data/4.json
data/6.json
data/10.json
data/13.json
data/12.json
data/2.json
data/5.json
data/9.json
data/0.json
data/1.json
data/8.json
data/14.json
data/16.json


### json에서 text 형태소 추출

In [12]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    text = bs(article["text"],"lxml").text
    
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "text_pos": ["%s_%s" % (first, second) for first, second in mecab.pos(text)],
            "pk": article["pk"]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train = labeled_train.set_index('pk')

labeled_train

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000


,istroll,text_pos
pk,,
76063,False,"[이석기_NNP, 와_JC, 경기_NNG, 동부_NNG, 가_JKS, 지난_VV+E..."
76064,True,"[.._SY, 그냥_MAG, 옆_NNG, 에_JKB, 앉_VV, 은_ETM, 할아버..."
76065,False,"[프로_NNG, 야구팀_NNG, 이나_JC, 축구_NNG, 농구_NNG, 배구_NN..."
76066,False,"[오래_MAG, 전_NNG, 에_JKB, 가입_NNG, 했_XSV+EP, 는데_EF..."
76067,False,"[누가_NP+JKS, 더_MAG, 이뻐_VA+EF, ?_SF]"
76068,False,"[황_NNP, 크라테스_NNP, 란_VCP+ETM, 별명_NNG, 을_JKO, 가진..."
76069,False,"[안녕_NNG, 한다_XSV+EC, 농_NNG, 여러분_NP, !_SF, 잘_MAG..."
76070,False,"[ㄲ_UNKNOWN, ㅑ_UNKNOWN, ~~~~~~~~_SY, 계란_NNG, 말_..."
76071,False,"[제_NP, 가_JKS, 생각_NNG, 하_XSV, 는_ETM, 것_NNB, 은_J..."


###bag of words

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)

    train_data_features = vectorizer.fit_transform((labeled_train[col_name].apply(lambda x:" ".join(x)))).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    labeled_train = pd.concat([labeled_train, pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)],axis=1)
    
    return labeled_train

##LDA

In [14]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    for i in range(0,num):
        if i%10000 == 0: print(i)
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = index
    
    train = pd.concat([train,df],axis=1)
        
    return train

In [15]:
max_features = 1000
#%time labeled_train = make_lda(labeled_train, 5000, 1000, "text_pos")
%time labeled_train = make_bag_of_words(labeled_train, 1000, "text_pos")

CPU times: user 34.4 s, sys: 3.72 s, total: 38.2 s
Wall time: 38.3 s


### predictor, model 생성

In [16]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['text_pos', label])

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [17]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

0.685503494312


##Result
  * text의 morphs를 bag of words로 변환 (feature : 1,000개) - 0.637526911627
  * text의 pos를 bag of words로 변환 (feature : 1,000개) - 0.661265083065
  * 데이터를 셔플하고 text의 pos를 bag of words로 변환 (feature : 1,000개) - 0.688875771784
  
  * text의 pos를 lda로 변환 (keep_n : 5,000개, num_topics : 1,000개) - 0.643143549702
  * bs의 파라미터를 "lxml"이 아닌 "html.parser"로 주고 pos를 bag of words로 변환 (feature : 1,000개) - 0.657726939833

In [18]:
import pickle

pickle.dump(labeled_train[pre], open("text_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)